In [ ]:
#hide 
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp graphtools

# Graph tools

> Functions for general graph manipulation and property checking

In [ ]:
#export 
from nbdev.showdoc import *
import networkx as nx
import warnings
import scipy.sparse as sp

## Utilities

In [ ]:
#export
def non_pendant_edges(G):
    "Returns all non pendant edges of a graph `G`"
    edges = list(G.edges())
    edges = [edge for edge in edges if not is_pendant(G, edge)]
    return edges

def is_pendant(G, edge):
    "Checks if `edge` is pendant in the graph `G`"
    if G.degree(edge[0]) == 1 or G.degree(edge[1]) == 1:
        return True
    else:
        return False

def has_isolated_nodes(G):
    "Checks if the graph `G` has isolated nodes"
    if len(list(nx.isolates(G))) > 0:
        return True
    else:
        return False
    
def edges_removed(G, Gp):
    "If Gp is made by removing edges from G this method will return a list of those edges"
    return list(set(G.edges()) - set(Gp.edges()))

## Spectral

The normalised Laplacian of a graph $\mathcal{G}$ is given by 
$$\mathcal{L}(\mathcal{G}) = \begin{cases}
1 & \text{if }u=v \text{ and } d_u > 0 \\
\frac{-1}{\sqrt{d_ud_v}} & \text{if }u \sim v \\
0 & otherwise
\end{cases}
$$

In this convention the diagonal entry is always $1$ unless the graph has isolated nodes in which case the entries are $0$ at those nodes. By setting `setdiag=True` then the diagonal entries will always be $1$ (even in the case of isolated nodes)

In [ ]:
#export 
def laplacian(G, setdiag=False):
    "Laplacian matrix of the graph `G`"
    L = nx.normalized_laplacian_matrix(G)
    if setdiag:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            L.setdiag(1)
    return L

In [ ]:
#hide 

## tests
import networkx as nx
import numpy as np

G = nx.barabasi_albert_graph(100, 2)
G.add_edge(0, 1) # the initial condition of BA(n, 2) means it can have pendant edges, this stops that happening
G_with_pendant = G.copy()
G_with_pendant.add_node(100)
G_with_pendant.add_edge(0, 100)
G_with_isolate = G.copy()
G_with_isolate.add_node(100)

# testing non_pendant_edges
assert non_pendant_edges(G) == list(G.edges())
assert non_pendant_edges(G_with_isolate) == list(G_with_isolate.edges())
assert non_pendant_edges(G_with_pendant) != list(G_with_pendant.edges())
assert set(G_with_pendant.edges()) - set(non_pendant_edges(G_with_pendant)) == {(0, 100)}

# testing is_pendant
assert is_pendant(G_with_pendant, (0, 100))
assert is_pendant(G_with_pendant, (100, 0))
assert np.sum([is_pendant(G, edge) for edge in G.edges()]) == 0
assert np.sum([is_pendant(G_with_isolate, edge) for edge in G.edges()]) == 0
assert np.sum([is_pendant(G_with_pendant, edge) for edge in G_with_pendant.edges()]) == 1

# testing has_isolated_nodes
assert not has_isolated_nodes(G)
assert not has_isolated_nodes(G_with_pendant)
assert has_isolated_nodes(G_with_isolate)

# testing edges_removed
G_with_edge = nx.Graph()
G_with_edge.add_nodes_from([0, 1])
G_no_edge = G_with_edge.copy()
G_with_edge.add_edge(0, 1)
assert edges_removed(G_with_edge, G_no_edge) == [(0, 1)]
assert edges_removed(G_no_edge, G_with_edge) == []

# testing laplacian
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (1, 3)])
L = laplacian(G)
L_manual = np.eye(4)
L_manual[0][1] = L_manual[1][0] = -1/np.sqrt(3) #edge 0 -- 1
L_manual[1][2] = L_manual[2][1] = -1/np.sqrt(6) #edge 1 -- 2
L_manual[1][3] = L_manual[3][1] = -1/np.sqrt(6) #edge 1 -- 3
L_manual[2][3] = L_manual[3][2] = -1/np.sqrt(4) #edge 2 -- 3
assert np.allclose(L_manual, L.todense())

# isolated node case
G.remove_edge(0, 1)
L = laplacian(G)
L_manual[0][0] = 0
L_manual[0][1] = L_manual[1][0] = 0 #edge 0 -- 1
L_manual[1][2] = L_manual[2][1] = -1/np.sqrt(4) #edge 1 -- 2
L_manual[1][3] = L_manual[3][1] = -1/np.sqrt(4) #edge 1 -- 3
assert np.allclose(L_manual, L.todense())

# isolated node but with diag set to 1
L = laplacian(G, setdiag=True)
L_manual[0][0] = 1
assert np.allclose(L_manual, L.todense())

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_metrics.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.
